In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

<IPython.core.display.Javascript object>

In [11]:
IF_SQLITE = True
from CoreConfig.CoreConfig import CoreConfig
if IF_SQLITE:
    from DBUtility.SQLiteUtility import DBSQLiteConnection, CreateVenturesDealLeadsIfNotExist, InsertVenturesDealLeadsRecord
    from DBUtility.SQLiteUtility import FetchVenturesDealLeadsRecordAll, FetchVenturesPerson
else:
    from DBUtility.DBUtility import DBConnection, CreateVenturesDealLeadsIfNotExist, InsertVenturesDealLeadsRecord
    from DBUtility.DBUtility import FetchVenturesDealLeadsRecordAll, FetchVenturesPerson
from pandas import DataFrame
import datetime
CoreConfig.initialize_core_config()
if IF_SQLITE:
    DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
else:
    DBConnection().configure_connection_information(**CoreConfig().get_db_connection_info())

CreateVenturesDealLeadsIfNotExist().execute()

Initialize  Ventures Deals Leads table            failed. Please Check your connection
table nab_ventures_deals_leads already exists


In [5]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box
from StylingUtility.StylingUtility import CustomStyling

In [23]:
formtitle = widgets.HTML("<center><H3>Ventures Deals Lead Form</H3><center>")
df_out = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})
with df_out:
    df = DataFrame(FetchVenturesDealLeadsRecordAll().execute(), columns=['Person_name', 'Deals_leads_date', 'Number_of_leads'])
    display(df)
    
person_list = FetchVenturesPerson().execute()
person_dict = {v[0]:v[0] for i, v in enumerate(person_list)}
person = widgets.Dropdown(
                options= person_dict,
                description='Select your person:',
                layout=CustomStyling.box_layout,
                style=CustomStyling.style
              )

deals_leads_date = widgets.DatePicker(
                    value = datetime.datetime.now().date(),
                    description='Pick a Date',
                    layout= CustomStyling.box_layout,
                    style = CustomStyling.style)

number_of_leads=widgets.IntText(
        value=0,
        description='Number of leads:',
        layout= CustomStyling.box_layout,
        style = CustomStyling.style
        )



button = widgets.Button(
           description='Submit the form',
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_submit_form_button_clicked(b):
    InsertVenturesDealLeadsRecord(person.value, deals_leads_date.value, number_of_leads.value ).execute()
    df = DataFrame(FetchVenturesDealLeadsRecordAll().execute(), columns=['Person_name', 'Deals_leads_date', 'Number_of_leads'])
    df_out.clear_output()
    with df_out:
        display(df)
button.on_click(on_submit_form_button_clicked)

In [24]:
def form(person, deals_leads_date, number_of_leads):

    return person, deals_leads_date, number_of_leads

In [25]:
ui = widgets.VBox(children=[formtitle,
                            df_out,
                            person,
                            deals_leads_date,
                            number_of_leads,
                           button], layout=CustomStyling.items_layout)

In [26]:
out = interactive_output(form, {'person':person,
                                'deals_leads_date':deals_leads_date,
                                'number_of_leads' : number_of_leads
                               })
display(ui, out)


Output()